In [1]:
import requests
import numpy as np
import json
import os
import tensorflow as tf

In [2]:
def scale_data(x):
    nom = (x-min_x)*2
    denom = max_x - min_x
    denom[denom==0] = 1
    return -1 + nom/denom

def get_data():
    resp = requests.get('http://vm05.htl-leonding.ac.at/data/all?max={}'.format(time_steps))
    if resp.status_code != 200:
        raise ApiError('GET /data/all?max={} {}'.format(time_steps, resp.status_code))
    x = np.empty([time_steps, 5])
    for i, item in enumerate(resp.json()['object']):
        if not all([sensor in item for sensor in sensors]):
            return np.array([])
        x[i] = np.array([item[sensor] for sensor in sensors])
    return x[np.newaxis, :]

In [3]:
sensors = ['co2Indoor', 'humidityIndoor', 'humidityOutdoor', 'temperatureIndoor', 'temperatureOutdoor']

with open(os.path.join('../../finalModels/smallGRU', 'config.json')) as json_data_file:
    data = json.load(json_data_file)
batch_size = int(data['batch_size'])
time_steps = int(data['time_steps'])
min_x = np.array(data['min_x'])
max_x = np.array(data['max_x'])

In [4]:
x = get_data()
x.shape

(1, 50, 5)

In [5]:
x_scaled = scale_data(x)
x_scaled.shape

(1, 50, 5)

In [9]:
x_scaled = np.vstack([x_scaled, x_scaled, x_scaled, x_scaled])
x_scaled.shape

(4, 50, 5)

In [10]:
model = tf.keras.models.load_model(os.path.join('../../finalModels/smallGRU', 'model.h5'))

In [13]:
preds = (model.predict(x_scaled, batch_size=batch_size) > 0.5).astype("int32").flatten()
preds

array([0, 0, 0, 0], dtype=int32)